In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ggplot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,r2_score,mean_squared_log_error
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [ ]:
train=pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')
test=pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')
sample_submission=pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv')


In [ ]:
train.head(20)

In [ ]:
train=train.loc[~(train['date_time']=='2011-01-01 00:00:00')].reset_index(drop=True)


In [ ]:
train.info()

In [ ]:
test.head()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.describe().T

In [ ]:
all_data = pd.concat([train, test])
# convert to datatime format
all_data['date_time'] = pd.to_datetime(all_data['date_time'])
all_data.head()

In [ ]:
all_data.tail()

In [ ]:
all_data.shape

In [ ]:
import math

def pb_add(X):
    X['day'] = X.date_time.dt.weekday
    is_odd = (X['sensor_4'] < 646) & (X['absolute_humidity'] < 0.238)
    X['is_odd'] = is_odd
    diff = X['date_time'] - min(X['date_time'])
    trend = diff.dt.days
    X['f1s'] = np.sin(trend * 2 * math.pi / (365 * 1)) 
    X['f1c'] = np.cos(trend * 2 * math.pi / (365 * 1))
    X['f2s'] = np.sin(2 * math.pi * trend / (365 * 2)) 
    X['f2c'] = np.cos(2 * math.pi * trend / (365 * 2)) 
    X['f3s'] = np.sin(2 * math.pi * trend / (365 * 3)) 
    X['f3c'] = np.cos(2 * math.pi * trend / (365 * 3)) 
    X['f4s'] = np.sin(2 * math.pi * trend / (365 * 4)) 
    X['f4c'] = np.cos(2 * math.pi * trend / (365 * 4)) 
    X['fh1s'] = np.sin(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 1))
    X['fh1c'] = np.cos(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 1))
    X['fh2s'] = np.sin(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 2))
    X['fh2c'] = np.cos(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 2))
    X['fh3s'] = np.sin(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 3))
    X['fh3c'] = np.cos(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 3))
    
    sensor_features = [
        'deg_C', 
        'relative_humidity', 'absolute_humidity', 
        'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5' ]
    
    lags = [-1, -4, -24, -7 * 24]  # last 4 hours, last 2 days, last 4 weeks
    for sensor_feature in sensor_features:
        this = X[sensor_feature]
        # look back
        for lag in lags:
            feature = f'{sensor_feature}_{abs(lag)}b'
            this_f = X[sensor_feature].shift(lag)
            X[feature] = (this_f - this).fillna(0)
        # look forwards
        for lag in lags:
            feature = f'{sensor_feature}_{abs(-lag)}f'
            this_f = X[sensor_feature].shift(-lag)
            X[feature] = (this_f - this).fillna(0)
            
    return X

In [ ]:
# The months will be used for folds split
months = all_data["date_time"].dt.month[:len(train)]
## New idea
all_data["hour"] = all_data["date_time"].dt.hour
all_data["working_hours"] =  all_data["hour"].isin(np.arange(8, 21, 1)).astype("int")
all_data["is_weekend"] = (all_data["date_time"].dt.dayofweek >= 5).astype("int")
all_data['hr'] = all_data.date_time.dt.hour*60+all_data.date_time.dt.minute
all_data['satday'] = (all_data.date_time.dt.weekday==5).astype("int")
all_data["SMC"] = (all_data["absolute_humidity"] * 100) / all_data["relative_humidity"]
all_data.drop(columns = 'hour', inplace = True)
all_data

In [ ]:
# The months will be used for folds split
months = all_data["date_time"].dt.month[:len(train)]
## New idea
all_data["hour"] = all_data["date_time"].dt.hour
all_data["working_hours"] =  all_data["hour"].isin(np.arange(8, 21, 1)).astype("int")
all_data["is_weekend"] = (all_data["date_time"].dt.dayofweek >= 5).astype("int")
all_data['hr'] = all_data.date_time.dt.hour*60+all_data.date_time.dt.minute
all_data['satday'] = (all_data.date_time.dt.weekday==5).astype("int")
all_data["SMC"] = (all_data["absolute_humidity"] * 100) / all_data["relative_humidity"]
all_data.drop(columns = 'hour', inplace = True)


# convert datetime to timestamp(s)
# included::: all_data['time'] = all_data['date_time'].astype(np.int64)//10**9

all_data = pb_add(all_data.copy())

all_data.drop(columns = 'date_time', inplace = True)

In [ ]:
all_data

In [ ]:
df_train=all_data.iloc[:len(train),:]

In [ ]:
df_train

In [ ]:
#sns.histplot?

In [ ]:
plt.figure(figsize=(20,10))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.5)
for i,trgt in enumerate(train.columns[1:]):
    plt.subplot(4,3,i+1)
    sns.histplot(train[trgt],kde=True,color='r') 


In [ ]:
#df_train=train.drop('date_time',axis=1)

In [ ]:
abs(df_train.corr()[['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']])

In [ ]:
sns.heatmap(abs(df_train.corr()[['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']]),center=0,annot=True, fmt=".3f",linewidths=.5)
sns.set(rc={'figure.figsize':(40,40)})

In [ ]:
df_train

In [ ]:
'''
import statsmodels.api as sm
x=df_train.iloc[:,:-3]
y1=df_train['target_carbon_monoxide']
y2=df_train['target_benzene']
y3=df_train['target_nitrogen_oxides']

lm1=sm.OLS(y1,x).fit()
lm2=sm.OLS(y2,x).fit()
lm3=sm.OLS(y3,x).fit() 

'''


In [ ]:
#lm1.summary().tables[1]

In [ ]:
#lm2.summary().tables[1]

In [ ]:
#lm3.summary().tables[1]

In [ ]:
#df_train.columns[:-3]

In [ ]:
#dum=abs(df_train.corr()[['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']])


In [ ]:

#plt.figure(figsize=(15,15))
#plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.5)
#for i,trgt in enumerate(['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']):
    
#    plt.subplot(2,2,i+1)
#    plt.title(trgt+' corr')
#    ax=sns.barplot(x=dum.index,y=trgt,data=dum)
#    ax.set_xticklabels(labels=ax.get_xticklabels(), rotation=90);
    
    

In [ ]:
#we dropped the following columns because of they have little effects on targets
#df_train=df_train.drop(['deg_C','relative_humidity','absolute_humidity'],axis=1)

#visualization of outlier of features
plt.figure(figsize=(200,75))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.5)

for i,col in enumerate(df_train.columns):

    plt.subplot(33,3,i+1)
    sns.boxplot(x=col,data=df_train,color='g')

In [ ]:
# function for cleaning outlier 
def cleanOutlier(data):
    
    for feature in data.columns:
         if feature not in ['target_carbon_monoxide', 'target_benzene','target_nitrogen_oxides']:
            q1=data[feature].quantile(0.25)
            q3=data[feature].quantile(0.75)
            iqr=q3-q1
            floor=q1-1.5*iqr
            ceiling=q3+1.5*iqr
            data[feature][data[feature]>ceiling]=ceiling
            data[feature][data[feature]<floor]=floor  
        
    return data

dum2=cleanOutlier(df_train.copy())

plt.figure(figsize=(10,10))
for i,col in enumerate(dum2.columns):

    plt.subplot(5,3,i+1)
    sns.boxplot(x=col,data=dum2,color='g')

In [ ]:
#now improving our own model

In [ ]:
df_train=dum2.copy()


In [ ]:
df_train

In [ ]:
sc=StandardScaler()


In [ ]:
X=df_train.copy().drop(['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'],axis=1)
df_test=all_data.iloc[len(train):,:].drop(['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'],axis=1)
y_tcm=df_train['target_carbon_monoxide']
y_tb=df_train['target_benzene']
y_tno=df_train['target_nitrogen_oxides']

In [ ]:
X

In [ ]:
df_test

In [ ]:
sc=StandardScaler()
X_=sc.fit_transform(X)
df_test=sc.transform(df_test)

tcm_train,tcm_val,y_tcm_train,y_tcm_val=train_test_split(X_,y_tcm,test_size=0.25,random_state=42)
tb_train,tb_val,y_tb_train,y_tb_val    =train_test_split(X_,y_tb, test_size=0.25,random_state=32)
tno_train,tno_val,y_tno_train,y_tno_val=train_test_split(X_,y_tno,test_size=0.25,random_state=52)

#tcm_train=sc.fit_transform(tcm_train)
#tcm_val=sc.transform(tcm_val)

#tb_train=sc.fit_transform(tb_train)
#tb_val=sc.transform(tb_val)

#tno_train=sc.fit_transform(tno_train)
#tno_val=sc.transform(tno_val)

print('tcm_train shape: {}, tcm_val shape: {}, y_tcm_train shape: {}, y_tcm_val shape: {}'.format(tcm_train.shape,tcm_val.shape,y_tcm_train.shape,y_tcm_val.shape))
print('tb_train shape:  {}, tb_val shape:  {}, y_tb_train shape:  {}, y_tb_val shape:  {}'.format(tb_train.shape,tcm_val.shape,y_tb_train.shape,y_tb_val.shape))
print('tno_train shape: {}, tno_val shape: {}, y_tno_train shape: {}, y_tno_valshape:  {}'.format(tno_train.shape,tno_val.shape,y_tno_train.shape,y_tno_val.shape))


# target_carbon_monoxide

In [ ]:
# from sklearn.metrics import SCORERS
# list(SCORERS.keys())


from sklearn.metrics import mean_squared_error,r2_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
params= { 
     'colsample_bytree': [0.4, 0.5,0.8],   # Ağac başına değişkenlerden alınacak olan örnek sayısı
     'max_depth': [2,3,4,20],
     'max_leaves':[5,10,20],
     'gamma':[0,0.25,10],
     'reg_alpha':[0.1,0.2,0.8],
}
model_tcm=xgb.XGBRegressor()
optimal_params=GridSearchCV(model_tcm,params,cv=3,n_jobs = -1, verbose = 2,scoring='neg_root_mean_squared_error').fit(tcm_train,y_tcm_train)
optimal_params.best_params_



In [ ]:
model_tcm=xgb.XGBRegressor(    seed=42,
                               learning_rate=0.1,
                               max_debth=5,
                               reg_lamba=0.1,
                               subsample=0.9,
                               max_leaves=10,
                               colsample_bytree=0.5,
                               n_estimators=100
                               ).fit(tcm_train,y_tcm_train,
                                     verbose=True,
                                     early_stopping_rounds=5,
                                     eval_metric='rmse',
                                     eval_set=[(tcm_val,y_tcm_val)])

model_tcm_train_pred=model_tcm.predict(tcm_train)
model_tcm_val_pred=model_tcm.predict(tcm_val)
model_test_pred=model_tcm.predict(df_test)


print('---target_carbon_monoxide---')

print('train_rmse : {}'.format(np.sqrt(mean_squared_error(model_tcm_train_pred,y_tcm_train))))
print('train_r2   : {}'.format(                  r2_score(model_tcm_train_pred,y_tcm_train)))
print('val_rmse   : {}'.format(np.sqrt(mean_squared_error(model_tcm_val_pred,y_tcm_val))))
print('val_r2     : {}'.format(                  r2_score(model_tcm_val_pred,y_tcm_val)))

print('RMLSE      : {}'.format(np.sqrt(mean_squared_log_error(model_tcm_val_pred,y_tcm_val))))




In [ ]:
residual=model_tcm_val_pred-y_tcm_val
df_dum=pd.DataFrame(np.vstack([model_tcm_val_pred,y_tcm_val,residual]).T,columns=['Pred','True','Residual'])
df_dum 

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(2,2,1)
plt.title('Pred and True')
sns.scatterplot(x='Pred',y='True',data=df_dum);
plt.subplot(2,2,2)
plt.title('Residual')
sns.scatterplot(x=df_dum.index,y='Residual',data=df_dum);

# target_benzene

In [ ]:
#grid.best_params_

In [ ]:
model_tb=xgb.XGBRegressor(   
                            reg_alpha =8,
                            colsample_bytree = 0.9,
                            max_leaves=5,
                            n_estimators=200,
                            learning_rate=0.1,
                            gamma=7,
                            max_depth=15 ,
                            random_state=42).fit(tb_train,y_tb_train,verbose=True)





model_tb_train_pred=model_tb.predict(tb_train)
model_tb_val_pred=model_tb.predict(tb_val)
model_test_pred=model_tb.predict(df_test)

print('---target_benzene---')
print('train_rmse : {}'.format(np.sqrt(mean_squared_error(model_tb_train_pred,y_tb_train))))
print('train_r2   : {}'.format(                  r2_score(model_tb_train_pred,y_tb_train)))
print('val_rmse  : {}'.format(np.sqrt(mean_squared_error(model_tb_val_pred,y_tb_val))))
print('val_r2    : {}'.format(                  r2_score(model_tb_val_pred,y_tb_val)))
print('RMLSE      : {}'.format(np.sqrt(mean_squared_log_error(model_tb_val_pred,y_tb_val))))

In [ ]:
residual=model_tb_val_pred-y_tb_val
dum2=pd.DataFrame(np.vstack([model_tb_val_pred,y_tb_val,residual]).T,columns=['pred','true','residual'])
dum2

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(2,2,1)
plt.title('pred and true')
sns.scatterplot(x='pred',y='true',data=dum2);
plt.subplot(2,2,2)
plt.title('Residual')
sns.scatterplot(x=dum2.index,y='residual',data=dum2);

# target_nitrogen_oxides

from sklearn.metrics import mean_squared_error,r2_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
model_params= { 
     'colsample_bytree': [0.4, 0.5,0.6],   # Ağac başına değişkenlerden alınacak olan örnek sayısı
     'max_depth': [2,3,4],
     'max_leaves':[5,10,20],
     'reg_alpha':[0.1,0.2,0.8],
}
model=xgb.XGBRegressor()
grid=GridSearchCV(model,model_params,cv=10,n_jobs = -1, verbose = 2).fit(tb_train,y_tb_train)
grid

In [ ]:

model_tno=xgb.XGBRegressor( reg_lambda=0.9,
                            colsample_bytree = 0.5,
                            max_leaves=8,
                            n_estimators=200,
                            learning_rate=0.1,
                            gamma=10,
                            max_depth=10 ,
                            random_state=42).fit(tno_train,y_tno_train,verbose=True)





model_tno_train_pred=model_tno.predict(tno_train)
model_tno_val_pred=model_tno.predict(tno_val)
model_test_pred=model_tno.predict(df_test)


print('---target_nitrogen_oxides---')
print('train_rmse : {}'.format(np.sqrt(mean_squared_error(model_tno_train_pred,y_tno_train))))
print('train_r2   : {}'.format(                  r2_score(model_tno_train_pred,y_tno_train)))
print('val_rmse   : {}'.format(np.sqrt(mean_squared_error(model_tno_val_pred,y_tno_val))))
print('val_r2     : {}'.format(                  r2_score(model_tno_val_pred,y_tno_val)))

print('RMLSE      : {}'.format(np.sqrt(mean_squared_log_error(model_tno_val_pred,y_tno_val))))


In [ ]:
residual=model_tno_val_pred-y_tno_val
df_dum=pd.DataFrame(np.vstack([model_tno_val_pred,y_tno_val,residual]).T,columns=['Pred','True','Residual'])
df_dum

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(2,2,1)
plt.title('Pred and True')
sns.scatterplot(x='Pred',y='True',data=df_dum);
plt.subplot(2,2,2)
plt.title('Residual')
sns.scatterplot(x=df_dum.index,y='Residual',data=df_dum);

In [ ]:
sample_submission['target_carbon_monoxide']=model_tcm.predict(df_test)
sample_submission['target_benzene']=model_tb.predict(df_test)
sample_submission['target_nitrogen_oxides']=model_tno.predict(df_test)

In [ ]:
sample_submission

In [ ]:
sample_submission.to_csv('/kaggle/working/submission.csv', index=False)
